In [2]:
from trees import Tree
from wrappers import GPTneoX_DenseWrapper, ActivationWrapper

import torch
import numpy as np
import umap
import plotly.graph_objects as go

/Users/collinszczepanski/Code/cris/LLM/chenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
names = ['The lungs turn', 'The lungs suppy the', 'The lungs work by', 'The lungs'
         'The lungs use', 'The lungs are made of']

In [24]:
model = "EleutherAI/pythia-1.4b-deduped"
wrapper = ActivationWrapper(model)

layer_num = -3
layer = wrapper.make_layer_wrapper(layer_num, 'mlp')

In [25]:
wrapper.batch_logits(names, tokens = 'all').shape

torch.Size([3, 5, 50278])

In [26]:
vocab_dict = wrapper.get_vocab()
reversed_dict = {v: k for k, v in vocab_dict.items()}

In [27]:
#all_acts, all_toks = layer.batch_activations(names)
last_acts, last_toks = layer.batch_activations(names, tokens='all')

print(last_acts.shape)

distances = torch.cdist(last_acts, last_acts, p=2)

torch.Size([3, 5, 2048])


In [34]:
num_tok_samples = 1
temp = 0.1
all_to_embed = []
colors = []

num_gens = 15

hold_sens = names

acts = []

for i in range(num_gens):
    thing, new_sens = wrapper.generate_and_prepare(hold_sens, num_tok_samples, temp)
    res, to = layer.batch_activations(thing, tokens='last', tokenized_prior=True)
    all_to_embed.extend(new_sens)
    blue = [i] * len(new_sens)
    colors.extend(blue)
    hold_sens = new_sens
    acts.append(res.detach().numpy())

print(all_to_embed)
print(colors)

colors = np.array(colors)
colors = colors.astype(float)
colors = (colors - colors.min()) / (colors.max() - colors.min())

acts = np.concatenate(acts, axis=0)
print(acts.shape)


['The lungs turn into', 'The lungs suppy the blood', 'The lungs work by breathing', 'The lungsThe lungs use oxygen', 'The lungs are made of two', 'The lungs turn into a', 'The lungs suppy the blood and', 'The lungs work by breathing in', 'The lungsThe lungs use oxygen to', 'The lungs are made of two types', 'The lungs turn into a giant', 'The lungs suppy the blood and oxygen', 'The lungs work by breathing in oxygen', 'The lungsThe lungs use oxygen to breathe', 'The lungs are made of two types of', 'The lungs turn into a giant sponge', 'The lungs suppy the blood and oxygen to', 'The lungs work by breathing in oxygen and', 'The lungsThe lungs use oxygen to breathe.', 'The lungs are made of two types of tissue', 'The lungs turn into a giant sponge,', 'The lungs suppy the blood and oxygen to the', 'The lungs work by breathing in oxygen and exh', 'The lungsThe lungs use oxygen to breathe. The', 'The lungs are made of two types of tissue:', 'The lungs turn into a giant sponge, absorbing', 'T

In [35]:
umapper = umap.UMAP(n_components=3)
emb = umapper.fit_transform(acts)
print(emb.shape)

x, y, z = emb[:, 0], emb[:, 1], emb[:, 2]
fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=4,
        color=colors,
        colorscale = 'viridis',
        opacity=0.8
    ),
    text=all_to_embed,
    hoverinfo='text'        
)])

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    margin=dict(l=0, r=0, b=0, t=0)
)

(75, 3)


/Users/collinszczepanski/Code/cris/LLM/chenv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

